# OntoGPT graph generation
We ran OntoGPT on the desiccation subset of our dataset; here, we'll post-process the output and format it as a graph.

In [1]:
import sys
sys.path.append('/mnt/home/lotrecks/haai_vanburen_lab/ontogpt/src/ontogpt/io')
from csv_wrapper import parse_yaml_predictions

/mnt/home/lotrecks/anaconda3/envs/ontogpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read in OntoGPT output

In [6]:
full_ent_df, full_rel_df = parse_yaml_predictions('../../data/ontogpt_output/destol_all_full_08May2024/output.txt', '../../knowledge_graph/schema/desiccation_full.yaml')

100%|█████████▉| 1209/1210 [00:00<00:00, 149267.44it/s]


TypeError: 'NoneType' object is not subscriptable

In [5]:
slim_ent_df, slim_rel_df = parse_yaml_predictions('../../data/ontogpt_output/destol_all_slim_08May2024/output.txt', '../../knowledge_graph/schema/desiccation.yaml')

100%|██████████| 929/929 [00:00<00:00, 156360.69it/s]


## Format into a networkx graph

In [7]:
slim_ent_df


,id,category,name,provided_by
0,AUTO:N/A,Gene,N/A,c92d3abf-f214-4576-9125-184a9e207613
1,AUTO:lipid%20peroxidation,Molecule,lipid peroxidation,c92d3abf-f214-4576-9125-184a9e207613
2,CHEBI:16856,Molecule,glutathione,c92d3abf-f214-4576-9125-184a9e207613
3,CHEBI:29191,Molecule,hydroxyl radical,c92d3abf-f214-4576-9125-184a9e207613
4,AUTO:Trichilia%20dregeana,Organism,Trichilia dregeana,c92d3abf-f214-4576-9125-184a9e207613
...,...,...,...,...
1556951,PR:000015781,Protein,ascorbic acid (ASC),fd92b661-e6fd-4359-8d35-2bb2670287ef
1556952,AUTO:total%20thiol,Molecule,total thiol,fd92b661-e6fd-4359-8d35-2bb2670287ef
1556953,NCBITaxon:56917,Organism,Dumortiera hirsuta,fd92b661-e6fd-4359-8d35-2bb2670287ef
1556954,AUTO:thylakoidal%20oligogalactolipids,Molecule,thylakoidal oligogalactolipids,fd92b661-e6fd-4359-8d35-2bb2670287ef
